In [1]:
# Imports
import sys
sys.path.append('C:\\Users\\mminbay\\Documents\\bait_research')
# from fasta_utils import sequences_from_fasta, sequences_to_fasta
# from utils import hamming_distance
from tqdm import tqdm
import os
import re
import time
from wfc_csp import wfc_csp, generate_randomized_strings
import random
from testing import get_test_info

In [2]:
files = [file for file in os.listdir('/home/mminbay/honors_thesis/sequence_data/synthesized/') if '.fasta' in file]

In [5]:
get_test_info(files[0])['pccov']

'70'

In [1]:
'joe'.replace('oe', 'k' + 'a' + 'm')

'jkam'

In [8]:
d = 40
ALPHABET = ['A', 'G', 'T', 'C']
l = 120
for k in [10, 100, 1000, 10000, 100000]:
    for i in range(3):
        solution = ''.join(random.choice(ALPHABET) for _ in range(l))
        strings = generate_randomized_strings(solution, k, d)
        start = time.time()
        s, dist = wfc_csp(strings, ['A', 'G', 'C', 'T'], d)
        end = time.time()
        print('Solution for {} strings found in {}. Max distance = {}'.format(k, end - start, dist))

Solution for 10 strings found in 0.0054280757904052734. Max distance = 32
Solution for 10 strings found in 0.005285739898681641. Max distance = 31
Solution for 10 strings found in 0.0052225589752197266. Max distance = 33
Solution for 100 strings found in 0.00748443603515625. Max distance = 35
Solution for 100 strings found in 0.0074198246002197266. Max distance = 37
Solution for 100 strings found in 0.00789332389831543. Max distance = 36
Solution for 1000 strings found in 0.03404378890991211. Max distance = 37
Solution for 1000 strings found in 0.0334780216217041. Max distance = 38
Solution for 1000 strings found in 0.03412961959838867. Max distance = 38
Solution for 10000 strings found in 0.29258203506469727. Max distance = 38
Solution for 10000 strings found in 0.2939329147338867. Max distance = 38
Solution for 10000 strings found in 0.2928659915924072. Max distance = 39
Solution for 100000 strings found in 2.92562198638916. Max distance = 40
Solution for 100000 strings found in 2.87

In [12]:
# Directories
SEQUENCES_DIR: str = 'C:\\Users\\mminbay\\Documents\\bait_research\\test_data'
CLUSTERS_DIR: str = 'C:\\Users\\mminbay\\Documents\\bait_research\\clusters'
OUTPUTS_DIR: str = 'C:\\Users\\mminbay\\Documents\\bait_research\\outputs'


In [58]:
def read_report(report_path: str):
    stupid_alignments = {}
    last_num = r"[0-9]+\.$"
    with open(report_path, 'r') as file:
        line = file.readline()
        while line:
            line = line.strip()
            if 'Identifier' in line:
                for i in range(2):
                    line = file.readline() # skip 2 lines 
                line = line.strip()
                rep = int(re.search(last_num, line).group()[:-1])
                for i in range(6):
                    line = file.readline() # skip 6
                line = line.strip()[1:-1]
                items = line.split(', ')
                items = [int(x) for x in items]
                stupid_alignments[rep] = items
            line = file.readline()
    return stupid_alignments  
            

In [31]:
def hamming_decide(str1, str2, m):
    '''
    '''
    distance = 0
    for i in range(len(str1)):
        if str1[i] == 'N': # character N does not match with anything
            distance += 1 
        elif str1[i] != str2[i]:
            distance += 1
        if distance > m:
            return False
    return True

In [102]:
# SETUP BLOCK
sequence_name = 'megares_100k_sequences.fasta'
_, seq = sequences_from_fasta(os.path.join(SEQUENCES_DIR, sequence_name))
seqlens = [len(x) for x in seq]
seq = ''.join(seq)
seq = seq.upper()
sa = divsufsort(seq)

Number of sequences read:  1
Total length:  100000


c:\Users\mminbay\AppData\Local\anaconda3\envs\bait_research\lib\site-packages\pydivsufsort\divsufsort.py:75: UserWarning: converting str argument uses more memory
  warnings.warn("converting str argument uses more memory")


In [105]:
def seed_and_extend(
    seq: str,
    bait: str,
    m: int,
    sa = None,
    seed_len: int = 10,
    seqlens = None,
):
    '''
    Applies the seed-and-extend heuristic to search for imperfect matches of the bait
    in the sequence. Returns a list of indices of the matches.

    Arguments:
        seq (str): Sequence to search for the baits in. 
        bait (str): Bait to search for.
        m (int): Allowed Hamming distance for each match.
        sa (suffix array): Suffix array of the string. If not provided, will be constructed.
        seed_len (int): Seed length to use for the heuristic. Every seed-length substring of the 
            bait will be used as seeds.
        seqlens (list[int]): If `seq` was obtained by concatenating sequences, this list 
            should have the lengths of each individual sequence before concatenation. This 
            is because baits should not be able to align around the concatenation points, so
            these values will be used to eliminate those baits.
    '''
    if sa is None:
        checkpoint = time.time()
        sa = divsufsort(seq)
        print('Divsufsort took {} seconds.'.format(time.time() - checkpoint))
    if seqlens is None:
        seqlens = [len(seq)]

    lb = len(bait)
    if seed_len > lb:
        raise Exception('Specified seed length is larger than the actual bait.')
    
    final_matches = set()
    checked_indices = set()

    lookup_time = 0
    discard_time = 0
    alignment_time = 0
    d_picked = 0
    d_checked = 0
    d_concat = 0
    d_negative = 0
    
    # for i in tqdm(range(lb - seed_len + 1), desc = 'Considering seeds', unit = 'seed'):
    for i in range(lb - seed_len + 1):
        checkpoint = time.time()
        seed = bait[i: i + seed_len]
        seed_matches = sa_search(seq, sa, seed)
        lookup_time += time.time() - checkpoint
        if seed_matches[1] is None:
            continue
        seed_matches = set(sa[seed_matches[1]: seed_matches[0] + seed_matches[1]])
        checkpoint = time.time()
        sum_seqlens = 0
        for seqlen in seqlens:
            discard_matches = set()
            sum_seqlens += seqlen
            for match in seed_matches:
                # find actual start of this alignment and check if it covers a concatenation
                actual_start = match - i
                if actual_start in final_matches: # if we already checked the coverage
                    discard_matches.add(match)     
                    d_picked += 1
                elif actual_start in checked_indices:
                    discard_matches.add(match)
                    d_checked += 1
                elif actual_start < 0:
                    discard_matches.add(match)
                    d_negative += 1
                elif actual_start < sum_seqlens and actual_start > sum_seqlens - lb:
                    discard_matches.add(match)
                    d_concat += 1
                checked_indices.add(actual_start)
            seed_matches = seed_matches.difference(discard_matches) # VERY INEFFICIENT - change
        discard_time += time.time() - checkpoint
        checkpoint = time.time()
        for match in seed_matches:
            actual_start = match - i
            sub = seq[actual_start: actual_start + lb]
            if len(sub) != len(bait):
                print(actual_start, 'wtf happened here')
            if hamming_decide(bait, sub, m):
                final_matches.add(actual_start)
        alignment_time += time.time() - checkpoint
    # print('Lookup time:', lookup_time)
    # print('Discard time:', discard_time)
    # print('Alignment time:', alignment_time)
    # print('Discarded because picked:', d_picked)
    # print('Discarded because checked:', d_checked)
    # print('Discarded because negative:', d_negative)
    # print('Discarded because concat:', d_concat)
    # print('Found alignments:', len(final_matches))
    return final_matches

In [85]:
_, baits = sequences_from_fasta(os.path.join(OUTPUTS_DIR, 'megares_100k_sequences_catch.fasta'))
stupid_alignments = read_report(os.path.join(OUTPUTS_DIR, 'megares_100k_comparison.txt'))

Number of sequences read:  347
Total length:  41640


In [114]:
non_exact = 0
no_align = 0
missed = 0
for bait in baits:
    bait = bait.upper()
    search_res = sa_search(seq, sa, bait)
    occurence = search_res[1]

    in_sa = sa[occurence]
    results = seed_and_extend(
        seq,
        bait,
        40,
        sa,
        seed_len = 20,
        seqlens = seqlens
    )
    if in_sa not in stupid_alignments.keys():
        no_align += 1
        continue
    stupid = set(stupid_alignments[in_sa])
    if stupid != results:
        # print(in_sa)
        # print(stupid)
        # print(results)
        non_exact += 1
        missed += len(stupid.difference(results))
print(non_exact)
print(no_align)
    



c:\Users\mminbay\AppData\Local\anaconda3\envs\bait_research\lib\site-packages\pydivsufsort\divsufsort.py:75: UserWarning: converting str argument uses more memory
  warnings.warn("converting str argument uses more memory")


143
30


In [115]:
print(missed)

810


In [5]:
synthesize_sequence(
    1000,
    3,
    (100, 100),
    0.5,
    5,
    verbose = True
)

Generated the following base repeats:
['GCATCTCATCATTGTGGCCTTTCAAACCGGGCACGTACCCAAGTTTTGCTGACGTAATACTAGTGACCCCTACATTATCTATCATGGCTTGGCGGTCGAG', 'TCCGCGCGCAATACGCCTTAGTCCGTGTGGTAAGAGATTACATAGTTTCCGGGCCAGCAGACAAGTGATCGTATAGTATTAGCATATGGCGCGTGTCATA', 'TGCTCTTTGACTGCGCGCGATATAATCTGTTACTCGACCGCGCTTTTACGAAGTATGGAAGATGCAGGGCCTACATGCCACGTTTGAAAACGTACAATCA']
Index 881 has been populated with the following:
GCATCTCATCATTATGGCCTTTCAAACCGGGCACGTACCCAAGTTTTGCTGACGTAATACTAGTGACCCCTACATTATTTATCATGGCTTGGCGGGCGAG
Index 152 has been populated with the following:
GCATCTCATCACTGTGGCCATTCAAACCAGGCACGTACCCAAGTTTTGCTGACGTAATACTAGTGACCCCTACATTATCTATCATGGCTTGGCGGTCGAG
Index 772 has been populated with the following:
GCATCTCATTATTATGGCCTTTCAAACCGGGCACGTACCCAAGTTTTTCTGACGTAATACTAGTGACCCCTACATTATCTATCATGGCTTGGCGGTCGAG
Index 12 has been populated with the following:
GCATCTCATCATTGTGGCCTTTCAAACCGGGCACGTACCCAAGTTTTGCTGACGTAATACTAGAGACCCCTACATTATCTATCATGGCTTGGCGGTCGAA
Index 693 has been populated with the following:
T

'GTCACTTAAATAGCATCTCATCATTGTGGCCTTTCAAACCGGGCACGTACCCAAGTTTTGCTGACGTAATACTAGAGACCCCTACATTATCTATCATGGCTTGGCGGTCGAAGGAGCTCGTTTTACAGTAAGCAACGGACTTGTCCACGAGCGCATCTCATCACTGTGGCCATTCAAACCAGGCACGTACCCAAGTTTTGCTGACGTAATACTAGTGACCCCTACATTATCTATCATGGCTTGGCGGTCGAGGGCAGTACAGCTTCTCTATAGTAAACGACTACCCGCACTTTTTCAGGCCTTTCGGAATATATTGAGAGAGGATGGATAATTTACCCCTAATGGGATTTTGCCACCCGCGGGGTATCCCTTCGCAAGGAATACCGTGGCCGCCACCCATTACGTGGTAATCCGAACAGTGACGCCGACTTGATGATATGCGCGGGTACGGACCGGGGGTATGGTCCCGCACAGACACCAAAGATTCCTTCAATTGAGGGCTGTCCGGAGCGGAGTATCGCATGCGCACAGCCGCTGGGACTACTCAAACTGTACGGGCATATTGAAAGACCTCAATTACTCCCCTCAGGAAAATAGGGTAGGTGCGCTAAATTTATAAGAGTGACGCATTGGCCTCGAACCGTGTTGCCTAGTTTAATAGGCAAATGCGGATCGTCAGTTCCCAGCAAGGGTTCCGAGCGTAATACGCCTTAGTCCGTGTGGTAAGAGATTACATAGTTTCCGGGCCAGCAGACAAGTGATCGTACAGTATTAGCATATGGCGCGTGGCATATCAAACCGGGCACGTACCCAAGTTTTTCTGACGTAATACTAGTGACCCCTACATTATCTATCATGGCTTGGCGGTCGAGGATGAAAAAGCATCTCATCATTATGGCCTTTCAAACCGGGCACGTACCCAAGTTTTGCTGACGTAATACTAGTGACCCCTACATTATTTATCATGGCTTGGCGGGCGAGTGGTCATCGTGAGTTTCT

In [13]:
l = 4
m = 1
seq = 'AACTGGAGGTACCTGGAG'

In [14]:
baits = [seq[i:i +l] for i in range(len(seq) - l + 1)]

In [15]:
def hamming_distance(str1, str2):
    # Check if the strings are of equal length
    if len(str1) != len(str2):
        raise ValueError("Input strings must be of equal length")

    # Calculate Hamming distance
    distance = sum(c1 != c2 for c1, c2 in zip(str1, str2))

    return distance

In [16]:
coverages = {}
for bait in baits:
    covered_indices = []
    for i in range(len(seq) - l + 1):
        snippet = seq[i: i + l]
        if hamming_distance(snippet, bait) <= m:
            covered_indices.append(i)
    coverages[bait] = covered_indices

In [17]:
coverages

{'AACT': [0, 10],
 'ACTG': [1, 11],
 'CTGG': [2, 12],
 'TGGA': [3, 13],
 'GGAG': [4, 14],
 'GAGG': [5],
 'AGGT': [6],
 'GGTA': [7],
 'GTAC': [8],
 'TACC': [9],
 'ACCT': [0, 10],
 'CCTG': [1, 11]}

In [21]:
coli3 = [file for file in os.listdir('/home/mminbay/honors_thesis/outputs/megares/') if 'fasta' in file]

In [22]:
len(coli3)

20